# Exploring Hacker News Posts
# Đâu là các để tăng tương tác cho bài viết trên trang Hacker News
## Mục tiêu dự án: 
> * Trong dự án này, chúng ta sẽ khám phá dữ liệu trang Hacker News, giai đoạn 2015 - 2016, có thể tải tại [đây](https://dq-content.s3.amazonaws.com/356/hacker_news.csv)
> * Bộ dữ liệu này bao gồm các thông tin về những bài viết được đăng trên trang Hacker News, gồm các thuộc tính như Id, tiêu đề ( title ), Url, điểm số ( num_points ), số lượng bình luận ( num_comments ), tác giả  ( author ), thời gian đăng bài ( created_at )
> * Chúng ta sẽ đặc biệt chú ý đến các title dạng Ask HN : ..., show HN : ... và trả lời câu hỏi trong hai dạng tiêu đề này, <b> đâu là dạng tiêu đề nhận  nhiều sự chú ý hơn , Thời gian cụ thể nào mà đăng bài nhận được nhiều chú ý hơn </b>

Liên quan đến các vấn đề thu hút người xem thì lượng bình luận (num_comments) là yếu tố cần phải phân tích trọng tâm

In [1]:
import csv
with open("hacker_news.csv") as opened_file:  # Đọc file hacker_news.csv
    file = list(csv.reader(opened_file, delimiter = ","))

In [2]:
for row in file[:5] : # in 5 hàng đầu tiên của danh sách
    print(row)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


# Làm sạch dữ liệu

## 1. Xóa bỏ trùng lặp, header

In [9]:
HN = list()
hn = list()
for i in file[1:]: 
    if i[0] not in HN:
        HN.append(i[0])
        hn.append(i)

In [10]:
# So sánh số phần tử ở cả hai 
print(len(hn))
print(len(file)) 

20100
20101


#### Như vậy không có trùng lặp ở trong danh sách các bài post trên Hacker News

## 2. Lọc ra các bài viết có title dạng Ask HN, Show HN
Mục tiêu: Lọc ra các bài viết có title Ask HN, show HN vào từng list khác nhau

In [14]:
ask_hn = list()
show_hn = list()
other_hn = list()

for row in hn:   # Duyệt qua từng bản ghi trong hn
    title = row[1].lower()   # đưa title về dạng in thường để không bị bỏ sót các bài viết viết hoa khác nhau nhưng biểu thị cùng ý tưởng
    if title.startswith("ask hn"): # dùng method startswith() để lọc ra các bài post có tài title bắt đầu bằng ask hn hoặc show hn
        ask_hn.append(row)
        
    elif title.startswith("show hn"):
        show_hn.append(row)
        
    else:
        other_hn.append(row)

In [17]:
print(len(ask_hn))
print(len(show_hn))
print(len(other_hn))
print(len(ask_hn) + len(show_hn) + len(other_hn))

1744
1162
17194
20100


# Phân tích
### Trong hai thể loại title ask hn... và show hn ..., đâu là thể loại thu hút người dùng tương tác hơn ?

In [27]:
# Tính trung bình số comment trong các bài viết ask hn, show hn
total_cmt_ask_hn = 0  
total_cmt_show_hn = 0

for row in ask_hn:
    row[4] = int(row[4])
    total_cmt_ask_hn += row[4]  # Tính tổng số comment
for row in show_hn:
    row[4] = int(row[4])
    total_cmt_show_hn += row[4]

# Tính số comment trung bình
avg_ask_hn = total_cmt_ask_hn / len(ask_hn)    # Tính trung bình lượng comment trên 1 bài viết ask-hn
avg_show_hn = total_cmt_show_hn / len(show_hn)   # Tính trung bình lượng comment trên 1 bài viết show-hn

In [25]:
print(avg_ask_hn)
print(avg_show_hn)
print(round(avg_ask_hn/avg_show_hn, 2))

14.038417431192661
10.31669535283993
1.36


> * #### Như vậy lượng bình luận trung bình mỗi bài viết có title Ask HN: ... nhiều hơn bài viết có title Show HN khoảng 36%
> * #### Dường như các title Ask HN thu hút người đọc comment để trả lời các câu hỏi, giải thích, trình bày quan điểm cá nhân hơn là các bài viết Show HN mang tính chất cung cấp thông tin và biểu đạt câu chuyện

## Đâu là thời điểm đăng bài để các bài viết ask HN thu hút nhiều lượng tương tác nhất ??

In [30]:
import datetime as dt    # Sử dụng module datetime để làm việc với thời gian
comment_list = list()
for row in ask_hn:
    time = dt.datetime.strptime(row[6], "%m/%d/%Y %H:%M")
    hour = time.hour
    new_list = [hour, row[4]] # Tạo list của list trong có mỗi list con gồm 2 phần: Lượng comment của bài thứ i, giờ đăng bài của bài thứ i
    comment_list.append(new_list)

# Tạo hai dictionary
counts_by_hour = dict()  # Lưu thông tin số bài viết ở trong mỗi giờ
comment_by_hour = dict()   # Lưu thông tin tổng số comment trong các bài viết ở giờ đó
for ele in comment_list:
    if ele[0] not in counts_by_hour:   # Nếu chưa có giờ đó trong từ điển
        counts_by_hour[ele[0]] = 1       # Thiết lập giá trị ban đầu
        comment_by_hour[ele[0]] = ele[1]
    else:        # Nếu tồn tịa giờ đó trong từ điển
        counts_by_hour[ele[0]] += 1   # Cập nhật giá trị
        comment_by_hour[ele[0]] += ele[1]

In [31]:
print(counts_by_hour)
print(comment_by_hour)

{9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}
{9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}


### Tính lượng comment trung bình mỗi giờ

In [35]:
avg_cmt_hour = dict()
for key, value in counts_by_hour.items():
    comment_total = comment_by_hour[key]
    avg = comment_total / value
    avg_cmt_hour[key] = round(avg, 2)
for key, value in avg_cmt_hour.items():
    print(f"{key} : {value} ")

9 : 5.58 
13 : 14.74 
10 : 13.44 
14 : 13.23 
16 : 16.8 
23 : 7.99 
12 : 9.41 
17 : 11.46 
15 : 38.59 
21 : 16.01 
20 : 21.52 
2 : 23.81 
18 : 13.2 
3 : 7.8 
5 : 10.09 
19 : 10.8 
1 : 11.38 
22 : 6.75 
8 : 10.25 
4 : 7.17 
0 : 8.13 
6 : 9.02 
7 : 7.85 
11 : 11.05 


In [43]:
sorted_avg_cmt_hour = sorted(avg_cmt_hour.items(), reverse = True , key=lambda kv: (kv[1], kv[0]))
for ele in sorted_avg_cmt_hour:
    print(f"{ele[0]} : {ele[1]} ")

15 : 38.59 
2 : 23.81 
20 : 21.52 
16 : 16.8 
21 : 16.01 
13 : 14.74 
10 : 13.44 
14 : 13.23 
18 : 13.2 
17 : 11.46 
1 : 11.38 
11 : 11.05 
19 : 10.8 
8 : 10.25 
5 : 10.09 
12 : 9.41 
6 : 9.02 
0 : 8.13 
23 : 7.99 
7 : 7.85 
3 : 7.8 
4 : 7.17 
22 : 6.75 
9 : 5.58 


In [48]:
print("Top 5 hours which has the most average comments : ")
for ele in sorted_avg_cmt_hour[:5]:
    hour = dt.time(ele[0])
    print(f"{hour} has the average number of comments is {ele[1]} ") 

Top 5 hours which has the most average comments : 
15:00:00 has the average number of comments is 38.59 
02:00:00 has the average number of comments is 23.81 
20:00:00 has the average number of comments is 21.52 
16:00:00 has the average number of comments is 16.8 
21:00:00 has the average number of comments is 16.01 


#### Vì các thông tin của thời gian được tính theo múi giờ EST là GMT -5 , mà Việt Nam ở múi giờ GMT +7, do đó thời gian khi chuyển đổi sang Việt Nam là: 

In [55]:
print("Top 5 hours which has the most average comments : ")
for ele in sorted_avg_cmt_hour[:5]:
    hour = dt.datetime(year =1999,month = 1, day = 1 , hour = ele[0]) + dt.timedelta(hours = 12)
    hour = hour.time()
    print(f"{hour} has the average number of comments is {ele[1]} ") 

Top 5 hours which has the most average comments : 
03:00:00 has the average number of comments is 38.59 
14:00:00 has the average number of comments is 23.81 
08:00:00 has the average number of comments is 21.52 
04:00:00 has the average number of comments is 16.8 
09:00:00 has the average number of comments is 16.01 


 #### Dựa vào kết quả trên, có thể thấy rằng với múi giờ miền Đông Mỹ ( EST, GMT -5) thì thời gian tốt nhất để đăng bài là thời điểm từ 15:00 đến 2:00 hôm sau, đây là khoảng thời gian rảnh rỗi của đa số nhân viên văn phòng, người lao động